In [63]:
import pandas as pd
import ollama
from collections import Counter

In [64]:
def analyze_feedback(text):
    """Analyze individual feedback using the mistral model."""
    prompt = f"""Analyze the following feedback:
    
    "{text}"
    
    Return output in the following structured format:

    Sentiment: (pos/neu/neg)
    Insight: (A very short insight summarizing the feedback)
    Key Factors:
    Neg:
    - (Short bullet points in one or two words each listing negative aspects)
    Pos:
    - (Short bullet points in one or two words each listing positive aspects)
    Neu:
    - (Short bullet points in one or two words each listing neutral aspects, or 'N/A' if none)
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [ ]:
df = pd.read_csv("feedbacks.csv")
df = df.head(50)

In [66]:
if "FEEDBACK" not in df.columns:
    raise ValueError("The CSV file must contain a 'FEEDBACK' column.")

In [67]:
# Initialize counters
sentiment_counts = Counter()
all_positive = []
all_negative = []
all_neutral = []

In [68]:
# Process each feedback entry
for feedback in df["FEEDBACK"].dropna():
    analysis = analyze_feedback(feedback)

    # Extract sentiment safely
    sentiment_start = analysis.find("Sentiment:")
    insight_start = analysis.find("Insight:")
    neg_start = analysis.find("Neg:")
    pos_start = analysis.find("Pos:")
    neu_start = analysis.find("Neu:")

    sentiment = analysis[sentiment_start+10: insight_start].strip() if sentiment_start != -1 else "Unknown"

    # Extract key factors safely
    neg_section = analysis[neg_start+5:pos_start].strip() if neg_start != -1 and pos_start != -1 else "N/A"
    pos_section = analysis[pos_start+5:neu_start].strip() if pos_start != -1 and neu_start != -1 else "N/A"
    neu_section = analysis[neu_start+5:].strip() if neu_start != -1 else "N/A"

    sentiment_counts[sentiment] += 1

    if neg_section != "N/A":
        all_negative.extend([point.strip("- ") for point in neg_section.split("\n") if point.strip()])
    if pos_section != "N/A":
        all_positive.extend([point.strip("- ") for point in pos_section.split("\n") if point.strip()])
    if neu_section != "N/A" and "N/A" not in neu_section:
        all_neutral.extend([point.strip("- ") for point in neu_section.split("\n") if point.strip()])

In [69]:
# Determine overall sentiment
overall_sentiment = sentiment_counts.most_common(1)[0][0]

In [70]:
# Generate summary report
report = f"""
Overall Sentiment: {overall_sentiment}

Overall Insight: The collected feedback highlights both positive aspects and areas for improvement. Key concerns include {', '.join(set(all_negative))}, while strengths include {', '.join(set(all_positive))}.

Key Factors:
Neg:
- {', '.join(set(all_negative))}

Pos:
- {', '.join(set(all_positive))}

Neu:
- {', '.join(set(all_neutral)) if all_neutral else 'N/A'}
"""

In [71]:
# Print final report
print(report)


Overall Sentiment: neg

Overall Insight: The collected feedback highlights both positive aspects and areas for improvement. Key concerns include Abstract concepts explained without practical context, Lack of examples, Lack of engaging storytelling, Many doubts unresolved, Pace was too fast, Session duration is too long, Limited number of examples, Concepts felt too theoretical and abstract, Limited coverage of advanced topics, Monotone delivery, Repetitive, Abstract concepts lack context, Too few examples, Attention dropped towards the end, Incomplete explanation of real-world applications, Session too short, Some key points were glossed over, Redundancy, Outdated information, Simplicity of examples, Overlap between sections, Lack of enthusiasm, Lack of time for questions, while strengths include Examples provided were helpful, Extending the Q&A segment, Implementing an online forum for follow-up questions, Knowledgeable, Breaking sessions into smaller modules would improve focus, Inf

In [73]:
import pandas as pd
import ollama
from collections import Counter

def analyze_feedback(text):
    """Analyze individual feedback using the mistral model."""
    prompt = f"""
    Analyze the following feedback:

    "{text}"

    Return output in EXACTLY this structured format:

    Overall Sentiment: (Positive/Neutral/Negative)

    Overall Insight: 
    (One short paragraph summarizing key strengths and weaknesses)

    Key Factors:
    
    Negative:
    - (Short bullet points listing negative aspects)
    
    Positive:
    - (Short bullet points listing positive aspects)
    
    Neutral:
    - (Short bullet points listing neutral aspects, or 'N/A' if none)
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

# Load CSV file
df = pd.read_csv("feedbacks.csv")
df = df.head(5)

# Ensure the FEEDBACK column exists
if "FEEDBACK" not in df.columns:
    raise ValueError("The CSV file must contain a 'FEEDBACK' column.")

# Initialize sentiment counters and lists for key factors
sentiment_counts = Counter()
all_positive = []
all_negative = []
all_neutral = []

# Process each feedback entry
for feedback in df["FEEDBACK"].dropna():
    analysis = analyze_feedback(feedback)
    
    # Split sections properly
    try:
        sentiment = analysis.split("Overall Sentiment:")[1].split("\n")[0].strip()
        insight = analysis.split("Overall Insight:")[1].split("\nKey Factors:")[0].strip()
        
        negative = analysis.split("Negative:")[1].split("Positive:")[0].strip().split("\n- ")
        positive = analysis.split("Positive:")[1].split("Neutral:")[0].strip().split("\n- ")
        neutral = analysis.split("Neutral:")[1].strip().split("\n- ")
        
        sentiment_counts[sentiment] += 1
        
        # Remove empty or bad splits
        all_negative.extend([point.strip() for point in negative if point.strip()])
        all_positive.extend([point.strip() for point in positive if point.strip()])
        all_neutral.extend([point.strip() for point in neutral if point.strip() and "N/A" not in point])
    
    except IndexError:
        continue  # Skip bad format responses

# Determine overall sentiment
overall_sentiment = sentiment_counts.most_common(1)[0][0]

# Generate summary report
report = f"""
Overall Sentiment: {overall_sentiment}

Overall Insight:  
{insight}

Key Factors:

Negative:
- {', '.join(set(all_negative))}

Positive:
- {', '.join(set(all_positive))}

Neutral:
- {', '.join(set(all_neutral)) if all_neutral else 'N/A'}
"""

# Print final report
print(report)


Overall Sentiment: Negative

Overall Insight:  
The session was generally informative but felt somewhat repetitive. The participant appreciated the information provided, but suggested improvements to make it more concise and effective for learning retention.

   Key Factors:

   Negative:
   - Redundancy in content
   - Overlap between sections

   Positive:
   - Informative content
   - Potential for improved learning through summarization at the end

   Neutral:
   - N/A

Key Factors:

Negative:
- - Redundancy in content
   - Overlap between sections, - The session duration was excessive, causing a drop in attention towards the end., - The Q&A session was too brief and did not sufficiently address participant's concerns.
   - Not all doubts were resolved during the session., - Quantity of examples provided was insufficient
   - Abstract concepts lacked practical context, - The practical examples were too simple
   - The real-world applications were not fully explained
   - Advanced 